### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
from datetime import datetime 

# Read Ironore_price_market_index_test.csv File and store into Pandas data frame
commodity_to_load = "Resources/Ironore_price_market_index_test.csv"
purchase_data = pd.read_csv(commodity_to_load)
#purchase_data

In [2]:
#remove empty columns, fix naming error and bring Date in line with other dataframes
clean_data = purchase_data[['Date','Iron Price','Silver Price','Gold Price',' Copper Price','S&P GSCI Price']]
clean_data = clean_data.rename(columns={' Copper Price': 'Copper_Price',
                                       'Iron Price':'Iron_Price',
                                        'Silver Price':'Silver_Price',
                                        'Gold Price':'Gold_Price',
                                       'S&P GSCI Price':'S&P_GSCI_Price'})
clean_data['Date'] = clean_data['Date'].str[2:]
clean_data['Date'] = pd.to_datetime(clean_data['Date']).dt.to_period('M')
clean_data = clean_data[(clean_data['Date'] >= '2007-01')
                           & (clean_data['Date'] <= '2019-06')]
clean_data
#save data to Commodities_clean.csv
clean_data.to_csv('Outputs/Commodities_clean.csv')

,Date,Iron_Price,Silver_Price,Gold_Price,Copper_Price,S&P_GSCI_Price
84,2007-01,36.63,13.36,650.50,13.36,"5,498.77"
85,2007-02,36.63,14.31,664.20,14.31,"5,735.06"
86,2007-03,36.63,13.35,661.75,13.35,"5,920.55"
87,2007-04,36.63,13.50,677.00,13.50,"5,904.26"
88,2007-05,36.63,13.25,659.10,13.25,"5,812.55"
...,...,...,...,...,...,...
229,2019-02,83.43,15.82,"1,319.15",15.82,"2,493.11"
230,2019-03,86.37,15.10,"1,295.40",15.10,"2,533.23"
231,2019-04,93.79,14.99,"1,282.30",14.99,"2,605.41"
232,2019-05,97.76,14.48,"1,295.55",14.48,"2,391.42"


In [3]:
iron_data = clean_data[['Date','Iron_Price']]


In [4]:
#https://www.kaggle.com/docs/api
#https://stackoverflow.com/questions/64393520/r-not-working-as-lineterminator-within-python-csv-writer
# Read all_perth_310121.csv File and store into Pandas data frame
perth_to_load = "Resources/all_perth_310121.csv"

perth_data = pd.read_csv(perth_to_load)


In [5]:
#fix up Date_sold format errors and bring in line with other datasets
perth_data["DATE_SOLD"] = perth_data["DATE_SOLD"].str.replace('\r','')
perth_data["DATE_SOLD"] = pd.to_datetime(perth_data["DATE_SOLD"], errors='coerce').dt.to_period('M')
perth_data = perth_data[(perth_data["DATE_SOLD"] >= '2007-01')
                           & (perth_data["DATE_SOLD"] <= '2019-06')]
perth_data

,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,1 Acorn Place,South Lake,565000,4,2,2.0,600,160,2003.0,18300,Cockburn Central Station,1800,2018-09,6164,-32.115900,115.842450,LAKELAND SENIOR HIGH SCHOOL,0.828339,NaN
1,1 Addis Way,Wandi,365000,3,2,2.0,351,139,2013.0,26900,Kwinana Station,4900,2019-02,6167,-32.193470,115.859554,ATWELL COLLEGE,5.524324,129.0
2,1 Ainsley Court,Camillo,287000,3,1,1.0,719,86,1979.0,22600,Challis Station,1900,2015-06,6111,-32.120578,115.993579,KELMSCOTT SENIOR HIGH SCHOOL,1.649178,113.0
3,1 Albert Street,Bellevue,255000,2,1,2.0,651,59,1953.0,17900,Midland Station,3600,2018-07,6056,-31.900547,116.038009,SWAN VIEW SENIOR HIGH SCHOOL,1.571401,NaN
4,1 Aman Place,Lockridge,325000,4,1,2.0,466,131,1998.0,11200,Bassendean Station,2000,2016-11,6054,-31.885790,115.947780,KIARA COLLEGE,1.514922,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33651,9C Gold Street,South Fremantle,1040000,4,3,2.0,292,245,2013.0,16100,Fremantle Station,1500,2016-03,6162,-32.064580,115.751820,CHRISTIAN BROTHERS' COLLEGE,1.430350,49.0
33652,9C Pycombe Way,Westminster,410000,3,2,2.0,228,114,NaN,9600,Stirling Station,4600,2017-02,6061,-31.867055,115.841403,JOHN SEPTIMUS ROE ANGLICAN COMMUNITY SCHOOL,1.679644,35.0
33653,9D Pycombe Way,Westminster,427000,3,2,2.0,261,112,NaN,9600,Stirling Station,4600,2017-02,6061,-31.866890,115.841418,JOHN SEPTIMUS ROE ANGLICAN COMMUNITY SCHOOL,1.669159,35.0
33654,9D Shalford Way,Girrawheen,295000,3,1,2.0,457,85,1974.0,12600,Warwick Station,4400,2016-10,6064,-31.839680,115.842410,GIRRAWHEEN SENIOR HIGH SCHOOL,0.358494,NaN


In [6]:
perth_data.sort_values(by=["DATE_SOLD"])

,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
12225,22 Barang Circuit,Karawara,860000,4,2,2.0,529,252,2004.0,6700,Canning Bridge Station,2800,2007-01,6152,-32.007950,115.885610,CANNING COLLEGE,1.098032,NaN
302,1 Lanark Street,Menora,875000,2,1,2.0,660,421,1930.0,3600,Leederville Station,2600,2007-01,6050,-31.919750,115.856120,MOUNT LAWLEY SENIOR HIGH SCHOOL,1.297126,20.0
20969,4 Jarril Place,Koongamia,337500,3,1,2.0,1003,89,1959.0,18100,Midland Station,4100,2007-01,6056,-31.903610,116.041900,SWAN VIEW SENIOR HIGH SCHOOL,1.808833,NaN
4708,13 Pinaster Street,Menora,720000,3,1,4.0,711,111,1965.0,4100,Mount Lawley Station,2800,2007-01,6050,-31.915690,115.862440,MOUNT LAWLEY SENIOR HIGH SCHOOL,0.924620,20.0
19619,360 Lacey Street,Sawyers Valley,850000,3,1,3.0,14768,175,1920.0,32000,Midland Station,18000,2007-01,6074,-31.896610,116.191390,MUNDARING CHRISTIAN COLLEGE,1.075449,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7908,17 Brockwell Place,Hillarys,1075000,4,2,2.0,851,270,1990.0,20400,Whitfords Station,3900,2019-06,6025,-31.798360,115.741140,ST MARK'S ANGLICAN COMMUNITY SCHOOL,0.164422,58.0
11275,20 Parker Avenue,Sorrento,1380000,3,1,2.0,1031,131,1965.0,16900,Greenwood Station,3200,2019-06,6020,-31.830350,115.752650,SACRED HEART COLLEGE,0.663243,19.0
20759,4 Diver Cove,Yangebup,640000,4,2,3.0,638,270,2005.0,20000,Cockburn Central Station,5400,2019-06,6164,-32.125099,115.801163,DIVINE MERCY COLLEGE,2.261884,NaN
8017,17 Fauntleroy Street,Guildford,455000,4,2,2.0,655,168,1982.0,12900,East Guildford Station,181,2019-06,6055,-31.897740,115.979170,GUILDFORD GRAMMAR SCHOOL,0.485609,52.0


In [7]:
#Limit dataframe to variables of interest
clean_perth = perth_data[['SUBURB',
                          'PRICE',
                          'BEDROOMS',
                          'BATHROOMS',
                          'DATE_SOLD',
                          'POSTCODE']]
clean_perth = clean_perth.rename(columns={'DATE_SOLD': 'Date'})
#Add unique ID and add City for later merging with Sydney data
clean_perth.insert(0, 'ID', range(1, 1 + len(clean_perth)))
clean_perth['CITY'] = "Perth"
clean_perth

,ID,SUBURB,PRICE,BEDROOMS,BATHROOMS,Date,POSTCODE,CITY
0,1,South Lake,565000,4,2,2018-09,6164,Perth
1,2,Wandi,365000,3,2,2019-02,6167,Perth
2,3,Camillo,287000,3,1,2015-06,6111,Perth
3,4,Bellevue,255000,2,1,2018-07,6056,Perth
4,5,Lockridge,325000,4,1,2016-11,6054,Perth
...,...,...,...,...,...,...,...,...
33651,24879,South Fremantle,1040000,4,3,2016-03,6162,Perth
33652,24880,Westminster,410000,3,2,2017-02,6061,Perth
33653,24881,Westminster,427000,3,2,2017-02,6061,Perth
33654,24882,Girrawheen,295000,3,1,2016-10,6064,Perth


In [8]:
#merge Perth data with Commodities data 
Perth_commodities_df = pd.merge(clean_data, clean_perth, on="Date",how="inner")
Perth_commodities_df.head()

,Date,Iron_Price,Silver_Price,Gold_Price,Copper_Price,S&P_GSCI_Price,ID,SUBURB,PRICE,BEDROOMS,BATHROOMS,POSTCODE,CITY
0,2007-01,36.63,13.36,650.50,13.36,"5,498.77",232,Menora,875000,2,1,6050,Perth
1,2007-01,36.63,13.36,650.50,13.36,"5,498.77",3451,Menora,720000,3,1,6050,Perth
2,2007-01,36.63,13.36,650.50,13.36,"5,498.77",5552,Peppermint Grove,2100000,5,3,6011,Perth
3,2007-01,36.63,13.36,650.50,13.36,"5,498.77",8875,Walliston,627000,4,3,6076,Perth
4,2007-01,36.63,13.36,650.50,13.36,"5,498.77",8992,Karawara,860000,4,2,6152,Perth


In [9]:
#save data to perth_clean.csv
Perth_commodities_df.to_csv('Outputs/perth_housing.csv')

In [10]:
#Create Perth data frame with Iron ore prices
perth_iron_df = pd.merge(iron_data, clean_perth, on="Date",how="inner")
perth_iron_df.to_csv('Outputs/perth_iron_housing.csv')


In [11]:
# Read SydneyHousePrices.csv File and store into Pandas data frame
Sydney_to_load = "Resources/SydneyHousePrices.csv"
Sydney_data = pd.read_csv(Sydney_to_load)

In [12]:
#Add City indicator for when merged with Perth data
Sydney_data["CITY"] = "Sydney"
#sanity check to see earliest date
#Sydney_data.sort_values(by=['Date'])


In [13]:
#Change the Date from string to datetime then convert to desired format 
Sydney_data['Date'] = pd.to_datetime(Sydney_data['Date'], errors='coerce').dt.to_period('M')
Sydney_data

,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType,CITY
0,2019-06,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,Sydney
1,2019-06,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,Sydney
2,2019-06,3,Whale Beach,2107,2920000,3.0,3,2.0,house,Sydney
3,2019-05,4,Avalon Beach,2107,1530000,3.0,1,2.0,house,Sydney
4,2019-05,5,Whale Beach,2107,8000000,5.0,4,4.0,house,Sydney
...,...,...,...,...,...,...,...,...,...,...
199499,2014-06,199500,Illawong,2234,1900000,5.0,3,7.0,house,Sydney
199500,2014-05,199501,Illawong,2234,980000,4.0,3,2.0,house,Sydney
199501,2014-04,199502,Alfords Point,2234,850000,4.0,2,2.0,house,Sydney
199502,2013-09,199503,Illawong,2234,640000,3.0,2,2.0,townhouse,Sydney


In [14]:
#Rename columns matching Perth data to combine later
clean_Sydney = Sydney_data.rename(columns={'Id':'ID',
                          'suburb':'SUBURB',
                          'sellPrice':'PRICE',
                          'bed':'BEDROOMS',
                          'bath':'BATHROOMS',
                          'postalCode':'POSTCODE'})
clean_Sydney

,Date,ID,SUBURB,POSTCODE,PRICE,BEDROOMS,BATHROOMS,car,propType,CITY
0,2019-06,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,Sydney
1,2019-06,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,Sydney
2,2019-06,3,Whale Beach,2107,2920000,3.0,3,2.0,house,Sydney
3,2019-05,4,Avalon Beach,2107,1530000,3.0,1,2.0,house,Sydney
4,2019-05,5,Whale Beach,2107,8000000,5.0,4,4.0,house,Sydney
...,...,...,...,...,...,...,...,...,...,...
199499,2014-06,199500,Illawong,2234,1900000,5.0,3,7.0,house,Sydney
199500,2014-05,199501,Illawong,2234,980000,4.0,3,2.0,house,Sydney
199501,2014-04,199502,Alfords Point,2234,850000,4.0,2,2.0,house,Sydney
199502,2013-09,199503,Illawong,2234,640000,3.0,2,2.0,townhouse,Sydney


In [15]:
#merge commodity prices with Sydney data
sydney_commodities_df = pd.merge(clean_data, clean_Sydney, on="Date",how="inner")
#sydney_commodities_df
#save data to sydney_clean.csv
sydney_commodities_df.to_csv('Outputs/sydney_housing.csv')

In [16]:
#Create Sydney data frame with Iron ore prices
sydney_iron_df = pd.merge(iron_data, clean_Sydney, on="Date",how="inner")
sydney_iron_df.to_csv('Outputs/sydney_iron_housing.csv')
sydney_iron_df

,Date,Iron_Price,ID,SUBURB,POSTCODE,PRICE,BEDROOMS,BATHROOMS,car,propType,CITY
0,2007-01,36.63,33600,Bass Hill,2197,350000,3.0,2,1.0,house,Sydney
1,2007-01,36.63,35811,Ermington,2115,470000,2.0,1,1.0,house,Sydney
2,2007-01,36.63,35812,Ermington,2115,435000,3.0,2,2.0,house,Sydney
3,2007-01,36.63,36581,Rydalmere,2116,430000,4.0,2,2.0,house,Sydney
4,2007-01,36.63,36582,Rydalmere,2116,420000,3.0,1,2.0,house,Sydney
...,...,...,...,...,...,...,...,...,...,...,...
198825,2019-06,112.90,198944,Bangor,2234,775000,3.0,1,1.0,house,Sydney
198826,2019-06,112.90,198945,Menai,2234,740000,3.0,1,1.0,townhouse,Sydney
198827,2019-06,112.90,198946,Illawong,2234,1345000,4.0,3,3.0,house,Sydney
198828,2019-06,112.90,198947,Barden Ridge,2234,865000,3.0,2,2.0,house,Sydney


In [17]:
#combine Perth and Sydney data into one DataFrame
complete_city_df = pd.concat([clean_Sydney, clean_perth], ignore_index=True)
#limit variables to match eachother
complete_city_df = complete_city_df[[
                          'SUBURB',
                          'PRICE',
                          'BEDROOMS',
                          'BATHROOMS',
                          'Date',
                          'POSTCODE',
                           'CITY']]

#merge commodity prices with city data and add unique ID for each sale
complete_df = pd.merge(clean_data, complete_city_df, on="Date",how="inner")
complete_df.insert(0, 'ID', range(1, 1 + len(complete_df)))
complete_df

,ID,Date,Iron_Price,Silver_Price,Gold_Price,Copper_Price,S&P_GSCI_Price,SUBURB,PRICE,BEDROOMS,BATHROOMS,POSTCODE,CITY
0,1,2007-01,36.63,13.36,650.50,13.36,"5,498.77",Bass Hill,350000,3.0,2,2197,Sydney
1,2,2007-01,36.63,13.36,650.50,13.36,"5,498.77",Ermington,470000,2.0,1,2115,Sydney
2,3,2007-01,36.63,13.36,650.50,13.36,"5,498.77",Ermington,435000,3.0,2,2115,Sydney
3,4,2007-01,36.63,13.36,650.50,13.36,"5,498.77",Rydalmere,430000,4.0,2,2116,Sydney
4,5,2007-01,36.63,13.36,650.50,13.36,"5,498.77",Rydalmere,420000,3.0,1,2116,Sydney
...,...,...,...,...,...,...,...,...,...,...,...,...,...
223708,223709,2019-06,112.90,15.22,"1,409.00",15.22,"2,497.37",Victoria Park,594000,3.0,2,6100,Perth
223709,223710,2019-06,112.90,15.22,"1,409.00",15.22,"2,497.37",Bayswater,565000,3.0,1,6053,Perth
223710,223711,2019-06,112.90,15.22,"1,409.00",15.22,"2,497.37",Iluka,695000,4.0,2,6028,Perth
223711,223712,2019-06,112.90,15.22,"1,409.00",15.22,"2,497.37",Alfred Cove,557000,3.0,2,6154,Perth


In [18]:
#save data to complete_house_commodities.csv
complete_df.to_csv('Outputs/complete_house_commodities.csv')

In [19]:
iron_city_df = pd.concat([perth_iron_df, sydney_iron_df], ignore_index=True)
#limit variables to match eachother
iron_city_df = iron_city_df[['Date',
                             'Iron_Price',
                             'SUBURB',
                             'PRICE',
                             'BEDROOMS',
                             'BATHROOMS',
                             'POSTCODE',
                             'CITY']]
#iron_city_df
iron_city_df.to_csv('Outputs/complete_house_iron.csv')
iron_city_df

,Date,Iron_Price,SUBURB,PRICE,BEDROOMS,BATHROOMS,POSTCODE,CITY
0,2007-01,36.63,Menora,875000,2.0,1,6050,Perth
1,2007-01,36.63,Menora,720000,3.0,1,6050,Perth
2,2007-01,36.63,Peppermint Grove,2100000,5.0,3,6011,Perth
3,2007-01,36.63,Walliston,627000,4.0,3,6076,Perth
4,2007-01,36.63,Karawara,860000,4.0,2,6152,Perth
...,...,...,...,...,...,...,...,...
223708,2019-06,112.90,Bangor,775000,3.0,1,2234,Sydney
223709,2019-06,112.90,Menai,740000,3.0,1,2234,Sydney
223710,2019-06,112.90,Illawong,1345000,4.0,3,2234,Sydney
223711,2019-06,112.90,Barden Ridge,865000,3.0,2,2234,Sydney


In [20]:
#create data drame of 3 bedroom houses
three_bed_df = iron_city_df[iron_city_df['BEDROOMS']==3]
three_bed_df.to_csv('Outputs/three_bed_housing.csv')

In [21]:
#create data drame of 4 bedroom houses
four_bed_df = iron_city_df[iron_city_df['BEDROOMS']==4]
four_bed_df.to_csv('Outputs/four_bed_housing.csv')

In [22]:
#create data drame of 4 bedroom 2 bathroom houses
four_bed_2bath_df = four_bed_df[four_bed_df['BATHROOMS']==2]
four_bed_2bath_df.to_csv('Outputs/four_bed_two_bath_housing.csv')